In [2]:
from typing import TypedDict, List
from langgraph.graph import StateGraph, START, END
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [3]:
load_dotenv()

True

In [6]:
class AgentState(TypedDict):
    messages: List[HumanMessage]

In [5]:
llm = ChatOpenAI(model = "gpt-4o")

In [7]:
def process(state: AgentState) -> AgentState:
    response = llm.invoke(state["messages"])
    print(f"\n AI: {response.content}")
    return state

In [8]:
graph = StateGraph(AgentState)

graph.add_node("process_node", process)
graph.add_edge(START, "process_node")
graph.add_edge("process_node", END)

app = graph.compile()

In [10]:
user_input = input("Enter:")
result = app.invoke({"messages": [HumanMessage(content=user_input)]})

print(result)


 AI: I'm just a computer program, so I don't have feelings, but I'm here and ready to help you with whatever you need! How can I assist you today?
{'messages': [HumanMessage(content='how are you doing?', additional_kwargs={}, response_metadata={})]}


In [11]:
import os
from typing import TypedDict, List, Union
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv

In [12]:
load_dotenv()

True

In [13]:
class AgentState(TypedDict):
    messages: List[Union[HumanMessage, AIMessage]]

In [ ]:
llm = ChatOpenAI(model="gpt-4o")

In [14]:
def process(state: AgentState) -> AgentState:
    """ ddd"""
    response = llm.invoke(state["messages"])
    state["messages"].append(AIMessage(content=response.content))
    print(f"\n AI: {response.content}")

    return state

In [15]:
graph = StateGraph(AgentState)

graph.add_node("process_node", process)
graph.add_edge(START, "process_node")
graph.add_edge("process_node", END)

app = graph.compile()

conversation_history = []

In [19]:
user_input = input("Enter: ")
conversation_history.append(HumanMessage(content=user_input))
result = app.invoke({"messages": conversation_history})
print(f"result:{result}")
conversation_history = result["messages"]
print(f"History conversation: {conversation_history}")


 AI: Your name is Jerry.
result:{'messages': [HumanMessage(content='Hi, my name is Jerry', additional_kwargs={}, response_metadata={}), AIMessage(content='Hello Jerry! How can I assist you today?', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is my name?', additional_kwargs={}, response_metadata={}), AIMessage(content='Your name is Jerry.', additional_kwargs={}, response_metadata={}), HumanMessage(content='What is Phenylephrine used for?', additional_kwargs={}, response_metadata={}), AIMessage(content="Phenylephrine is a medication commonly used as a decongestant to relieve nasal congestion caused by colds, allergies, or sinus infections. It works by constricting blood vessels in the nasal passages, which reduces swelling and congestion. Phenylephrine can be found in oral forms, such as tablets and syrups, as well as in nasal sprays and ophthalmic solutions for other uses like treating redness in the eyes. It's important to follow the dosage instructions and

In [25]:
lst = [1, 2, 3, 4, 5, 6, 7]
last_three = lst[: -1]
print(last_three)

[1, 2, 3, 4, 5, 6]


In [3]:
from typing import TypedDict, Annotated, Sequence
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, ToolMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

In [4]:
load_dotenv()

True

In [43]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages] # add_messages will automatically append messages to the list.

@tool
def add(a: int, b: int):
    """add two numbers"""
    return a + b

@tool
def substract(a: int, b: int):
    """Substract two numbers"""
    return a - b

@tool
def multiply(a: int, b: int):
    """Multiply two numbers"""
    return a * b

tools = [add, substract, multiply]

model = ChatOpenAI(model = "gpt-4o").bind_tools(tools)

In [44]:
def model_call(state: AgentState) -> AgentState:
    system_prompt = SystemMessage(
        content="You are my AI assistant, please answer my query to the best of your ability."
    )
    #print([system_prompt] + state["messages"])
    response = model.invoke([system_prompt] + state["messages"])
    #print(response)
    return AgentState({"messages": [response]})

In [45]:
def should_continue(state: AgentState) -> str:
    messages = state["messages"]
    last_massage = messages[-1]
    print(last_massage)
    if last_massage.tool_calls:
        return "continue"
    else:
        return "end"

In [46]:
graph = StateGraph(AgentState)

graph.add_node("our_agent", model_call)

tool_node = ToolNode(tools=tools)
graph.add_node("tool", tool_node)

graph.add_edge(START, "our_agent")

graph.add_conditional_edges(
    source="our_agent",
    path=should_continue,
    path_map={
        "continue": "tool",
        "end": END,
    },
)

graph.add_edge("tool", "our_agent")

app = graph.compile()

In [47]:
def print_stream(stream):

    for s in stream:
        message = s["messages"][-1]
        if isinstance(message,tuple):
            print(message)
        else:
            message.pretty_print()

In [48]:
input = {"messages": ["user", "add 40 + 4, then use the result multiply by 2. Finally, determin whether 'How are you doing?' is a clinical question"]}

print_stream(app.stream(input, stream_mode="values"))

================================ Human Message =================================

add 40 + 4, then use the result multiply by 2. Finally, determin whether 'How are you doing?' is a clinical question
content='' additional_kwargs={'tool_calls': [{'id': 'call_NNAQXVtHqrtuhstEsJRrd7Ik', 'function': {'arguments': '{"a":40,"b":4}', 'name': 'add'}, 'type': 'function'}], 'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 140, 'total_tokens': 157, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_cbf1785567', 'id': 'chatcmpl-CIo4UjQClavoOFxhNHmT12i1JBrQ1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='run--6b5f0d0f-3625-40d2-89fd-d4ac406061d7-0' tool_calls=[{'name': 'add', 'args': {'a': 40, 'b': 4}, 'id': 

In [14]:
from typing import TypedDict, Annotated, Sequence
from dotenv import load_dotenv
from langchain_core.messages import BaseMessage, ToolMessage, SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode

In [5]:
load_dotenv()

True

In [6]:
document_content = ""

In [7]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages] # add_messages will automatically append messages to the list.

In [8]:
@tool
def update(content: str) -> str:
    """Updates the document with the provided content"""
    global document_content
    document_content = content
    return f"Document has been updated successfully! The current content is:\n{document_content}"

def save(filename: str) -> str:
    """ 
    save the current document to a text file and finish the process

    Args:
        filename: Name for the text file.
    """
    global document_content
    if not filename.endswith(".txt"):
        filename = f"{filename}.txt"

    try:
        with open(filename,"w") as file:
            file.write(document_content)

        print(f"\n Document has been saved to: {filename}")

        return f"Document has been saved to: {filename}"
    except Exception as e:
        return f"Error saving document:{str(e)}"
    

tools = [update, save]


In [9]:
model = ChatOpenAI(model="gpt-4o").bind_tools(tools)

In [10]:
def our_agent(state: AgentState) -> AgentState:
    #global document_content
    system_prompt = SystemMessage(content=f"""
    You are Drafter, a helpful writing assistant. You are going to help the user update and modify documents.
    - If the user wants to update or modify content, use the 'update' tool with the complete updated content.
    - If the user wants to save and finish, you need to use the 'save' tool.
    - Make sure to always show the current document state after modifications.

    The current document content is: {document_content}                                                                                                                        
    """)

    if not state["messages"]:
        user_input = "I'am ready to help you update a document. What would you like to create?"
        user_message = HumanMessage(content=user_input)
    else:
        user_input = input("\nWhat would you like to do with the document?")
        print(f"\n USER:{user_input}")
        user_message = HumanMessage(content=user_input)

    all_messages = [system_prompt] + list(state["messages"]) + [user_message]

    response = model.invoke(all_messages)

    print(f"\n AI: {response.content}")

    if hasattr(response, "tool_calls") and response.tool_calls:
        print(f"""USING TOOLS: {[tc["name"] for tc in response.tool_calls]}""")

    return {"messages": list(state["messages"]) + [user_message, response]}

def should_continue(state: AgentState) -> str:
    """Determine if we should continue or end the conversation"""
    
    messages = state["messages"]

    if not messages:
        return "continue"
    
    for message in reversed(messages):
        if(isinstance(message, ToolMessage)) and "saved" in message.content.lower() and "document" in message.content.lower():
            return "end"
        
    return "continue"


def print_messages(messages):
    """Pretty printer"""

    if not messages:
        return
    
    for message in messages[-3:]:
        if isinstance(message, ToolMessage):
            print(f"Tool result: {message.content}")



In [11]:
graph = StateGraph(AgentState)

graph.add_node("agent", our_agent)
graph.add_node("tools", ToolNode(tools))

graph.add_edge(START, "agent")
graph.add_edge("agent", "tools")

graph.add_conditional_edges(
    source="tools",
    path=should_continue,
    path_map={
        "continue": "agent",
        "end": END
    }
)

app = graph.compile()

In [12]:
def run_document_agent():
    print("\n====Drafter====\n")

    state = AgentState(messages=[])

    for step in app.stream(state, stream_mode="values"):
        if "messages" in step:
            print_messages(step["messages"])

    print("\n==========Drafter end =================")

In [17]:
run_document_agent()


====Drafter====


 AI: It seems you want to make some changes to the document. Please let me know what updates or modifications you'd like to make to the current document.

 USER:I don't feel good today

 AI: If you'd like to update the document to reflect this, we can include a note mentioning you're unwell, and perhaps suggest rescheduling due to not feeling well. Would you like me to make that change?

 USER:so i need to send a message to my friend to tell him I can not have diner with him. Can you write me the message?

 AI: Certainly! Here's a message you could consider sending:

---

Hi [Friend's Name],

I hope you're doing well. I wanted to let you know that I'm not feeling well today and, unfortunately, won't be able to make it to our dinner. I'm really sorry for the sudden change of plans.

Let's definitely try to reschedule for another time when I'm feeling better. I appreciate your understanding.

Take care,

Jerry

---

Would you like to replace the current document conten